#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head(10)

C:\Users\bribe\AppData\Local\Temp\ipykernel_26728\182781903.py:1: DtypeWarning: Columns (6,8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.0,canceled,24,US,1145.0,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.0,failed,28,US,1851.0,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.0,successful,254,US,3796.0,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.0,successful,367,US,2670.0,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.0,failed,1,CA,0.708148,0.738225
5,1771789139,Room For Growth!,Couture,Fashion,USD,2016-05-02,2000.0,2016-04-11 18:15:00,6.0,failed,2,US,6.0,6.000000
6,1301627822,Build a mini Udemy with Laravel and Vuejs,Web,Technology,CAD,2018-01-11,3000.0,2017-12-12 01:37:26,155.0,live,7,CA,15.563996,123.811806
7,881336601,Applitizer,Software,Technology,HKD,2017-08-20,100000.0,2017-07-21 16:50:05,1000.0,failed,1,HK,128.08538,127.831467
8,1669374237,"Help I Am Wolves fund their first album ""ABCD""...",Music,Music,EUR,2017-12-04,1000.0,2017-10-27 16:55:47,1465.0,successful,46,BE,0.0,1720.210418
9,556821416,Eco Bamboo Underwear funding....GOT WOOD ?,Fashion,Fashion,GBP,2012-12-13,9500.0,2012-11-13 10:58:34,1212.0,failed,43,GB,1928.137119,1953.672808


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [5]:
df_ks['goal'] = pd.to_numeric(df_ks['goal'], errors='coerce')
df_ks['pledged'] = pd.to_numeric(df_ks['pledged'], errors='coerce')
df_ks['backers'] = pd.to_numeric(df_ks['backers'], errors='coerce')
df_ks['usd pledged'] = pd.to_numeric(df_ks['usd pledged'], errors='coerce')

df_ks = df_ks.dropna()
df_ks.head(10)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.0,canceled,24.0,US,1145.000000,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.0,failed,28.0,US,1851.000000,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.0,successful,254.0,US,3796.000000,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.0,successful,367.0,US,2670.000000,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.0,failed,1.0,CA,0.708148,0.738225
5,1771789139,Room For Growth!,Couture,Fashion,USD,2016-05-02,2000.0,2016-04-11 18:15:00,6.0,failed,2.0,US,6.000000,6.000000
6,1301627822,Build a mini Udemy with Laravel and Vuejs,Web,Technology,CAD,2018-01-11,3000.0,2017-12-12 01:37:26,155.0,live,7.0,CA,15.563996,123.811806
7,881336601,Applitizer,Software,Technology,HKD,2017-08-20,100000.0,2017-07-21 16:50:05,1000.0,failed,1.0,HK,128.085380,127.831467
8,1669374237,"Help I Am Wolves fund their first album ""ABCD""...",Music,Music,EUR,2017-12-04,1000.0,2017-10-27 16:55:47,1465.0,successful,46.0,BE,0.000000,1720.210418
9,556821416,Eco Bamboo Underwear funding....GOT WOOD ?,Fashion,Fashion,GBP,2012-12-13,9500.0,2012-11-13 10:58:34,1212.0,failed,43.0,GB,1928.137119,1953.672808


### Importer les données

In [6]:
collection.delete_many({})
documents = df_ks.rename(columns={"ID":"_id"}).to_dict(orient='records')
collection.insert_many(documents)

InsertManyResult([872782264, 1326492673, 1688410639, 156812982, 1835968190, 1771789139, 1301627822, 881336601, 1669374237, 556821416, 160394496, 1248488072, 895565490, 1165822076, 424612381, 1182908609, 206040356, 396454289, 1522195027, 1429288669, 1441999253, 1202442815, 2116917173, 2117684680, 350628546, 35969505, 1849047042, 295864582, 496463082, 1390262683, 1041882251, 1130940376, 186326896, 1025722498, 1493170888, 1007463174, 2143760539, 2062002375, 1183084584, 1977195934, 1035882318, 1021441525, 1691723287, 680665134, 1864901261, 519449839, 1875047349, 217543389, 1263082934, 764437823, 1341393677, 1149469807, 1968352897, 693501911, 397524601, 520797301, 1195741430, 2024072867, 223758456, 1594750207, 726485587, 260647442, 1043191090, 395755489, 830800632, 2113778157, 163783398, 2069080555, 1224888533, 1841566953, 449615827, 442102012, 1054276073, 1292948737, 2036356057, 56346579, 797794426, 1086953170, 955493948, 133917405, 1702541310, 87518909, 1957326840, 1593966557, 1029624752,

## Question 1  

In [8]:
cur = collection.find().sort("backers",-1).limit(5)
list(cur)

[{'_id': 1386523707,
  'name': 'Fidget Cube: A Vinyl Desk Toy',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-10-20',
  'goal': 15000.0,
  'launched': '2016-08-30 22:02:09',
  'pledged': 6465690.3,
  'state': 'successful',
  'backers': 154926.0,
  'country': 'US',
  'usd pledged': 13770.0,
  'usd_pledged_real': 6465690.3},
 {'_id': 1755266685,
  'name': 'The Veronica Mars Movie Project',
  'category': 'Narrative Film',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': '2013-04-13',
  'goal': 2000000.0,
  'launched': '2013-03-13 15:42:22',
  'pledged': 5702153.38,
  'state': 'successful',
  'backers': 91585.0,
  'country': 'US',
  'usd pledged': 5702153.38,
  'usd_pledged_real': 5702153.38},
 {'_id': 286165030,
  'name': 'Torment: Tides of Numenera',
  'category': 'Video Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2013-04-06',
  'goal': 900000.0,
  'launched': '2013-03-06 14:47:37',
 

## Question 2

In [9]:
cur = collection.find({"state":"successful"})
len(list(cur))

52998

## Question 3

In [10]:
cur = collection.aggregate([{"$group":{"_id":"$main_category", "NumberOfProjects":{"$sum":1}}}])
list(cur)

[{'_id': 'Art', 'NumberOfProjects': 11260},
 {'_id': 'Design', 'NumberOfProjects': 11911},
 {'_id': 'Games', 'NumberOfProjects': 13987},
 {'_id': 'Music', 'NumberOfProjects': 19657},
 {'_id': 'Crafts', 'NumberOfProjects': 3463},
 {'_id': 'Film & Video', 'NumberOfProjects': 24982},
 {'_id': 'Publishing', 'NumberOfProjects': 15721},
 {'_id': 'Comics', 'NumberOfProjects': 4139},
 {'_id': 'Photography', 'NumberOfProjects': 4176},
 {'_id': 'Fashion', 'NumberOfProjects': 8943},
 {'_id': 'Technology', 'NumberOfProjects': 12871},
 {'_id': 'Theater', 'NumberOfProjects': 4305},
 {'_id': 'Dance', 'NumberOfProjects': 1488},
 {'_id': 'Journalism', 'NumberOfProjects': 1807},
 {'_id': 'Food', 'NumberOfProjects': 9805}]

## Question 4

In [11]:
cur = collection.aggregate([{"$match":{"country":"FR", "launched":{"$lt":"2016-01-01 00:00:00"}}}, {"$group":{"_id":"$country", "NumberOfProjects":{"$sum":1}}}])
list(cur)

[{'_id': 'FR', 'NumberOfProjects': 330}]

## Question 5

In [12]:
cur = collection.find({"$and":[{"country":"US"}, {"goal":{"$gt":200000.0}}]})
len(list(cur))

1841

## Question 6 

In [13]:
collection.create_index([("name",  "text")])

'name_text'

In [14]:
collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [15]:
cur = collection.find({"$text":{"$search":"Sport"}})
len(list(cur))

316